In [21]:
import requests
import xml.etree.ElementTree as ET
import tarfile

import io
from io import BytesIO

import re
import os

import json
import fitz
import ast

import sqlite3
import hashlib

import time
from datetime import datetime, timedelta


In [22]:
from dotenv import load_dotenv
load_dotenv('Key.env')

True

In [23]:
from openai import OpenAI

client = OpenAI(
    #Creates a client class
        api_key=os.environ.get("API_KEY"),
    )


In [24]:
def is_valid_python_code(code_str):
    try:
        ast.parse(code_str)
        return True
    except SyntaxError:
        return False
    
def create_unique_hash(input_string, input_list):
    import hashlib
    list_string = str(input_list)
    combined_input = input_string + list_string
    hash_object = hashlib.sha256(combined_input.encode())
    unique_hash = hash_object.hexdigest()
    return int(unique_hash[:8], 16)  # Convert part of the hash to an integer

def generate_weekly_spans(year):
    start_date = datetime(year, 1, 1)
    
    weekly_spans = []
    while start_date.year == year:
        end_date = start_date + timedelta(days=6) 
        weekly_spans.append((start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')))
        start_date += timedelta(days=7)
    
    return weekly_spans

In [25]:
debug = False
def process_pdf(url):
    response = requests.get(url)

    if response.status_code != 200:
        raise Exception("Request Failed!")

    # Open the PDF file with PyMuPDF directly from the in-memory bytes
    pdf_document = fitz.open(stream=BytesIO(response.content), filetype='pdf')

    # Extract text from each page and store it in a string
    pdf_text = ""
    for page_num in range(len(pdf_document)):
        page = pdf_document[page_num]
        pdf_text += page.get_text()

    # Optionally, encode the string as UTF-8
    pdf_text_utf8 = pdf_text.encode('utf-8').decode('utf-8')

    # Close the PDF document
    pdf_document.close()
    return pdf_text_utf8

def prompt_gpt(pdf, prompt_txt, word_start, word_cutoff, format_needed):
    # No good solution, simply grabs the first few hundred characters and hopes all the authors are in that area, has already failed so far, let's improve on this later
    intro = pdf[word_start:word_cutoff]
    # Prompts gpt [possibly just hard code this solutoin, honestly just doing this for practice]

    valid_syntax = False
    loop_count = 0

    while(not valid_syntax):
        intro = pdf[word_start:word_cutoff]
        loop_count += 1
        if(loop_count == 3):
          #To prevent an infinite loop
          return False

        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": "Without any extra explanation text, " + prompt_txt + " " + intro,
                }
            ],
            model="gpt-4",
        )
        if(debug):
            print(chat_completion.choices[0].message.content)

        valid_syntax = (is_valid_python_code(chat_completion.choices[0].message.content)) or (not format_needed)

    #AST will automatically throw an error if GPT's response is deemed bad
    
    if(format_needed):
        return ast.literal_eval(chat_completion.choices[0].message.content)
    return chat_completion.choices[0].message.content

def grab_author_institutions(pdf):
    return prompt_gpt(pdf, "find all authors and corresponding instiutions, and return them in a dictionary, author key, insti",0,1500, True)

Unite the Code that pulls from the source with code that pushes it to the dump folder, also make it go on a week by week basis


In [ ]:
#Arxiv API interface URL
base_url = 'http://export.arxiv.org/api/query'

#Query Given
search_query = 'all:machine learning'

#Amount of Paper data to be dumped into each .json
max_results = 50

#Jump off point for searching, gets slow when this number gets large
start_index = 0  

#Artifical limit set for now, eventually it will scrape the whole of a query
total_papers = 200
papers_data = []

while len(papers_data) < total_papers:

    params = {
        'search_query': search_query,
        'start': start_index,
        'max_results': max_results,
        'sortBy': 'submittedDate',
        'sortOrder': 'descending'
    }
    
    response = requests.get(base_url, params=params)
    
    if response.status_code != 200:
        raise Exception("Failed to retrieve data" + response.status_code)

    root = ET.fromstring(response.content)
    
    for entry in root.findall('{http://www.w3.org/2005/Atom}entry'):
        abs_link = entry.find('{http://www.w3.org/2005/Atom}id').text
        title = entry.find('{http://www.w3.org/2005/Atom}title').text.strip()
        published = entry.find('{http://www.w3.org/2005/Atom}published').text
        authors = entry.findall('{http://www.w3.org/2005/Atom}author')
        author_list = [author.find('{http://www.w3.org/2005/Atom}name').text for author in authors]
        
        papers_data.append({
            'id': int(abs_link[21:].replace('.', '').replace('v', '')),
            'title': title,
            'published': published,
            'authors': author_list
        })
        
        if len(papers_data) >= total_papers:
            break

    start_index += max_results

In [20]:
json.dump(papers_data, open("dump/" + "firstdump", 'w'), indent=4)

In [27]:
conn = sqlite3.connect('arxiv-db.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS papers (
    id INTEGER PRIMARY KEY,
    title TEXT,
    published TEXT,
    authors TEXT
)
''')

with open("dump/firstdump", 'r') as read:
    data = json.load(read)

for paper in papers_data:

    entry['authors'] = ', '.join(entry['authors'])
    
    cursor.execute('''
    INSERT INTO papers (id, title, published, authors)
    VALUES (?, ?, ?, ?)
    ''', (entry['id'], entry['title'], entry['published'], entry['authors']))